# Document representation

In [2]:
import sklearn.feature_extraction # Módulo de sklearn para extracción de características
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import numpy as np # Biblioteca de manejo de datos vectoriales
import pandas as pd # Biblioteca de manejo de conjuntos de datos
import spacy.lang.es # Biblioteca de procesamiento de lenguaje natural
import matplotlib.pyplot as plt # Biblioteca de visualización

Cargamos el conjunto de datos del curso.

In [3]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Cargamos las particiones de entrenamiento y prueba

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset spanish_diagnostics downloaded and prepared to /home/vscode/.cache/huggingface/datasets/fvillena___spanish_diagnostics/default/0.0.0/45c176cea64580ea9631f78c2867a657ede368597681e5337e9f1c976e4e84ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Cargamos una lista de stopwords desde la biblioteca Spacy.

In [4]:
stopwords = spacy.lang.es.stop_words.STOP_WORDS # La biblioteca Spacy tiene una lista de stopwords en español

## Representación de documentos

Para poder trabajar con datos de texto, estos deben ser representados de una manera que pueda ser interpretada por los algoritmos de minería de texto. Típicamente se desea llegar a una matriz que tenga tantas filas como documentos tenga nuestro corpus y tantas columnas como características fueron extraídas desde el texto.

Revisaremos 2 métodos de extracción de características:

* Bag-of-words: Este método extrae la frecuencia de aparición de cada una de las palabras del documento y representa un documento como un vector de tantas dimensiones como palabras tenga el vocabulario.

* Term frequency - inverse document frequency (TF-IDF): Este método extrae la frecuencia de aparición de cada una de las palabras y la multiplica por el inverso de la frecuencia de aparición de la palabra en todos los documentos. También se representa cada documento como un vector de tantas dimensiones como palabras tenga el vocabulario.

Instanciamos un extractor de características Bag-of-words.

In [7]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    stop_words = list(stopwords), # Le pasamos la lista de stopwords para eliminarlas del vocabulario
    max_df = 0.05, # Eliminamos del vocabulario el 5% de palabras más frecuentes (stopwords específicas del corpus)
    min_df = 2 # Eliminamos del vocabulario las palabras que tienen una frecuencia menor a 2 (típicamente palabras malformadas)
)

Ajustamos el vectorizador sobre los textos del conjunto de prueba.

In [8]:
count_vectorizer.fit(spanish_diagnostics["train"]["text"])

CountVectorizer(max_df=0.05, min_df=2,
                stop_words=['dar', 'siempre', 'pocas', 'dicen', 'más', 'una',
                            'ante', 'al', 'ciertas', 'días', 'habrá', 'algunas',
                            'según', 'primero', 'primer', 'diferente', 'soy',
                            'qeu', 'será', 'siguiente', 'tiene', 'poner',
                            'otro', 'aquello', 'haciendo', 'quizás', 'indicó',
                            'debido', 'esos', 'próximos', ...])

Exploraremos cómo está representando nuestros documentos este vectorizador.

Este es un texto de ejemplo del corpus.

In [9]:
spanish_diagnostics["train"]["text"][69983]

'- CARIES DENTINARIA PROFUNDA/  - Fundamento Clínico APS: caries profunda en relacion de la pieza dental 4.5 Caries de la dentina'

In [10]:
def get_word_scores(text,vectorizer):
    """A partir de un texto y un vectorizador retorna los puntajes asignados a cada palabra del texto"""
    feature_names = list({k: v for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}.keys()) # Vocabulario
    doc = vectorizer.transform([text]) # Vectorizamos el texto de entrada
    idxs = np.argwhere(doc)[:,1] # Extraemos los índices donde sí hay palabras representadas
    words = [feature_names[i] for i in idxs] # Extraemos las palabras asociadas a los índices extraídos
    scores = np.array(doc.todense())[0][idxs] # Extraemos los puntajes asociadas a los índices extraídos
    return list(reversed(sorted(zip(words,scores),key=lambda tup: tup[1]))) # Retornamos una lista de palabras y puntajes

Vemos que nuestro vectorizador le dio más peso a la palabra caries de nuestro documento porque es la palabra más frecuente y a a un grupo de palabras les asignó el mismo puntaje 1 porque cada una aprece 1 vez.

In [11]:
get_word_scores(spanish_diagnostics["train"]["text"][69983],count_vectorizer)

[('caries', 3),
 ('profunda', 2),
 ('relacion', 1),
 ('fundamento', 1),
 ('dentinaria', 1),
 ('dentina', 1),
 ('dental', 1)]

Ajustamos un vectorizador que utiliza TF-IDF y lo ajustamos.

In [13]:
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(
    stop_words = list(stopwords),
    max_df = 0.05,
    min_df = 2
)
tfidf_vectorizer.fit(spanish_diagnostics["train"]["text"])

TfidfVectorizer(max_df=0.05, min_df=2,
                stop_words=['dar', 'siempre', 'pocas', 'dicen', 'más', 'una',
                            'ante', 'al', 'ciertas', 'días', 'habrá', 'algunas',
                            'según', 'primero', 'primer', 'diferente', 'soy',
                            'qeu', 'será', 'siguiente', 'tiene', 'poner',
                            'otro', 'aquello', 'haciendo', 'quizás', 'indicó',
                            'debido', 'esos', 'próximos', ...])

Podemos observar que caries sigue siendo la palabra con el mayor puntaje. Pero vemos que todas las palabras tienen un puntaje distinto, en donde destacamos que la palabra fundamento tiene el menor puntaje. Intuitivamente podemos darnos cuenta que esta representación es mejor porque la palabra fundamento no nos aporta mucha información en el documento.

In [14]:
get_word_scores(spanish_diagnostics["train"]["text"][69983],tfidf_vectorizer)

[('caries', 0.5841232685802328),
 ('profunda', 0.5052614123771014),
 ('dentina', 0.36362077674894117),
 ('relacion', 0.30054370920913476),
 ('dentinaria', 0.27911008594273823),
 ('dental', 0.25831112531154554),
 ('fundamento', 0.19062330472026245)]

Vectorizamos los textos de nuestros conjuntos de entrenamiento y prueba. con el método CountVectorizer.transform()

In [15]:
text_vectorized_train = count_vectorizer.transform(spanish_diagnostics["train"]["text"])
text_vectorized_test = count_vectorizer.transform(spanish_diagnostics["test"]["text"])
feature_names = list({k: v for k, v in sorted(count_vectorizer.vocabulary_.items(), key=lambda item: item[1])}.keys())

La forma de nuestra matriz es de (cantidad de documentos en el conjunto, tamaño del vocabulario)

In [16]:
text_vectorized_train.shape

(70000, 13565)

In [17]:
text_vectorized_test.shape

(30000, 13565)